In [ ]:
# This notebook extracts indicator data under hazard and normal conditions

In [1]:
import pandas as pd
import xarray as xr
# Read all indicators
# Read gridded data (datasets provided at request)
hwi_edo= xr.open_dataset('heatwave_EDO.nc')
spei3= xr.open_dataset('SPEI3.nc')
spei6= xr.open_dataset('SPEI6.nc')
spi3= xr.open_dataset('SPI3.nc')
spi6= xr.open_dataset('SPI6.nc')
spi12= xr.open_dataset('SPI12.nc')
tmin= xr.open_dataset('min_temp.nc')
prec3= xr.open_dataset('precip-3mon.nc')
prec6= xr.open_dataset('precip-6mon.nc')
hwi= xr.open_dataset('heatwave_SMHI.nc')
spei= xr.open_dataset('SPEI12.nc')
dfi= xr.open_dataset('Fidx.nc')
tmax= xr.open_dataset('max_temp.nc')
prec12= xr.open_dataset('precip-12mon.nc')
# Read point data and merge with station data
disc_stations=pd.read_excel('discharge_stations.xlsx')
ssi= pd.read_excel('SSI_all.xlsx')
ssi= ssi.dropna(subset=['SSI'])
ssi= ssi.sort_values(by='time')
ssi= pd.merge(disc_stations[['id', 'latitude', 'longitude']], ssi, left_on='id', right_on='station')
disc= pd.read_csv('discharge_data.csv')
disc = disc.dropna(subset=['value'])
disc=disc.sort_values(by='date')
disc.date=pd.to_datetime(disc.date)
disc=disc[disc['date']>='1921-01-01']
disc=disc[disc['date']<'2022-01-01']
disc= pd.merge(disc_stations[['id', 'latitude', 'longitude']], disc, left_on='id', right_on='station_id')
prec_stations= pd.read_excel('rain_stations.xlsx')
dprec= pd.read_csv('dayrain_data.csv')
dprec = dprec.dropna(subset=['value'])
dprec=dprec.sort_values(by='ref')
dprec.ref=pd.to_datetime(dprec.ref)
dprec=dprec[dprec['ref']>='1921-01-01']
dprec=dprec[dprec['ref']<'2022-01-01']
dprec= pd.merge(prec_stations[['id', 'latitude', 'longitude']], dprec, left_on='id', right_on='station_id')
disc_stations=pd.read_excel(discharge_stations.xlsx')
ssi3= pd.read_excel('SSI_all3.xlsx')
ssi3= ssi3.dropna(subset=['SSI'])
ssi3= ssi3.sort_values(by='time')
ssi3= pd.merge(disc_stations[['id', 'latitude', 'longitude']], ssi3, left_on='id', right_on='station')
ssi6= pd.read_excel('SSI_all6.xlsx')
ssi6= ssi6.dropna(subset=['SSI'])
ssi6= ssi6.sort_values(by='time')
ssi6= pd.merge(disc_stations[['id', 'latitude', 'longitude']], ssi6, left_on='id', right_on='station')

C:\Users\MarlonVieiraPassos\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\MarlonVieiraPassos\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\MarlonVieiraPassos\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [12]:
input_df=pd.read_excel('haz_input_all.xlsx')


C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\1786167751.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  input_df=pd.read_excel('flood_input_rain.xlsx')


In [5]:
import numpy as np
# create a grid of latitude and longitude coordinates
lat_grid = np.linspace(55.3376, 69.1763, 100) 
lon_grid = np.linspace(11, 24.113, 100)
# Mask out values outside country boundaries
from shapely.geometry import Polygon, Point
from cartopy.io import shapereader
import geopandas
def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]
# Mask from natural earth (more below)
# request data for use by geopandas
resolution = '50m'
category = 'cultural'
name = 'admin_0_countries'

shpfilename = shapereader.natural_earth(resolution, category, name)
df = geopandas.read_file(shpfilename)

# get geometry and polygon of a country
poly = [df.loc[df['ADMIN'] == 'Sweden']['geometry'].values[0]]
exts = [min(lon_grid)-1, max(lon_grid)+1, min(lat_grid)-1, max(lat_grid)+1]
msk = Polygon(rect_from_bound(*exts)).difference( poly[0].simplify(0.01) )
# create boolean mask where True=inside polygon
mask=np.zeros((len(lat_grid),len(lon_grid)),dtype=bool)
for i, y in enumerate(lat_grid):
    for j, x in enumerate(lon_grid):
        mask[i,j]=Point(x,y).within(msk)
# Find the latitude and longitude indices on land
land_indices = np.where(mask == False)
# Combine the latitude and longitude indices into pairs
land_lat_lon_indices = list(zip(land_indices[0], land_indices[1]))

In [6]:
input_df

,Event,Hazard,County,Municipality,Start,End,Lat,Lon
0,Vemhån (1955),Flood,Jämtland,Vemhån,1955-11-26,1955-11-26,62.312521,14.039143
1,Gåxsjö (1956),Flood,Jämtland,Gåxsjö,1956-05-09,1956-05-09,63.672206,15.104303
2,Kolsva (1977),Flood,Västmanland,Kolsva,1977-04-04,1977-04-04,59.599842,15.844001
3,Fagersta (1977),Flood,Västmanland,Fagersta,1977-04-29,1977-04-29,59.966667,15.833333
4,Virsbo (1977),Flood,Västmanland,Virsbo,1977-04-29,1977-04-29,59.871369,16.048057
...,...,...,...,...,...,...,...,...
317,Nybro (2010),Flood,Kalmar,Nybro,2010-11-24,2010-11-24,56.744491,15.907514
318,St Sigfrid (2010),Flood,Kalmar,St Sigfrid,2010-11-24,2010-11-24,57.709477,11.993476
319,Örsjö (2010),Flood,Kalmar,Örsjö,2010-11-24,2010-11-24,56.700803,15.748027
320,Orrefors (2010),Flood,Kalmar,Orrefors,2010-11-24,2010-11-24,56.839479,15.748617


In [86]:
import numpy as np
from tqdm.notebook import tqdm
# Extract data (only )
option='event' # extract data during event times (year or start to end)
#option='all' # extract all data for all years at point
#option2='year' # extract for entire year
option2='duration'
df_results=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
# Extract nearest data
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
def find_nearest_on_land(lat, lon, lats, lons, land_lat_lon_indices):
    dist=100000000
    for pair in land_lat_lon_indices:
        lat_idx=pair[0]
        lon_idx=pair[1]
        lat_diff = np.abs(lats[lat_idx] - lat)
        lon_diff = np.abs(lons[lon_idx] - lon)
        dist_p = np.sqrt(lat_diff ** 2 + lon_diff ** 2)
        if dist_p<dist:
            dist=dist_p
            lat_near=lats[lat_idx]
            lon_near=lons[lon_idx]
    return lat_near, lon_near
def find_nearest_time(array, value):
    array = np.asarray(array)
    value = np.asarray(value).astype('datetime64[ns]')
    idx = np.abs(array - value).argmin()
    return array[idx]
def linear_min_max_normalization(data):
    min_val = np.nanmin(data)
    max_val = np.nanmax(data)
    normalized_data = [(x - min_val) / (max_val - min_val) for x in data]
    return normalized_data
def extract_nearest_point(df, lat, lon, times, start, end, var_time):
    start_near= find_nearest_time(times, start)
    end_near= find_nearest_time(times, end)
    df_time= df[df[var_time]>=start_near]
    df_time= df_time[df_time[var_time]<=end_near]
    # It has to be filtered by time first
    lat_diff = np.abs(df_time['latitude'] - lat)
    lon_diff = np.abs(df_time['longitude'] - lon)
    dist = np.sqrt(lat_diff ** 2 + lon_diff ** 2)
    nearest_idx = dist.idxmin()
    nearest_point = df.loc[nearest_idx, :]
    station_near= nearest_point.id
    df_pt = df[df['id']==station_near]
    lat_near = df_pt.latitude.values[0]
    lon_near = df_pt.longitude.values[0]
    return df_pt, lat_near, lon_near
def extract_grid_pt(xarray, var, lat, lon, start, end, option):
    times= pd.to_datetime(xarray.time.values)
    lat_near, lon_near =  find_nearest_on_land(lat, lon, xarray.lat.values, xarray.lon.values, land_lat_lon_indices)
    year = pd.to_datetime(start).year
    start_year = pd.to_datetime(f'{year}-01-01')
    end_year = pd.to_datetime(f'{year}-12-31')
    if option2=='year':
        start_near= find_nearest_time(times, start_year)
        end_near= find_nearest_time(times, end_year)
    else:
        start_near= find_nearest_time(times, start)
        end_near= find_nearest_time(times, end)
    start_idx=np.where(np.array(times)==start_near)[0][0]
    end_idx=np.where(np.array(times)==end_near)[0][0]+1
    data_pt= xarray.sel(lat=lat_near, lon=lon_near)[var].values
    n_data_pt= linear_min_max_normalization(data_pt)
    if option=='event':
        n_data_pt= n_data_pt[start_idx:end_idx]
        data_pt= data_pt[start_idx:end_idx]
        times_pt= times[start_idx:end_idx]
    else:
        times_pt= times
    return n_data_pt, data_pt, times_pt, lat_near, lon_near
def extract_data_pt(df, var, var_time, lat, lon, start, end, option):
    times= df[var_time].values
    df_pt, lat_near, lon_near = extract_nearest_point(df, lat, lon, times, start, end, var_time)
    if var=='value':
        var_values= df_pt[var].values
        var_values[var_values==0]=np.nan
        df_pt[var]=var_values
    df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
    times= df_pt[var_time].values
    year = pd.to_datetime(start).year
    start_year = pd.to_datetime(f'{year}-01-01')
    end_year = pd.to_datetime(f'{year}-12-31')
    if option2=='year':
        start_near= find_nearest_time(times, start_year)
        end_near= find_nearest_time(times, end_year)
    else:
        start_near= find_nearest_time(times, start)
        end_near= find_nearest_time(times, end)
    start_idx=np.where(np.array(times)==start_near)[0][0]
    end_idx=np.where(np.array(times)==end_near)[0][0]+1
    if option=='event':
        df_pt= df_pt[start_idx:end_idx]
    n_data_pt=df_pt['norm'].values
    data_pt=df_pt[var].values
    times_pt=df_pt[var_time].values
    lat_near= df_pt['latitude'].values[0]
    lon_near= df_pt['longitude'].values[0]
    return n_data_pt, data_pt, times_pt, lat_near, lon_near

with tqdm(total=len(input_df)) as pbar:
    for n in range(len(input_df)):
        lat=input_df['Lat'].iloc[n]
        lon=input_df['Lon'].iloc[n]
        haz=input_df['Hazard'].iloc[n]
        event=input_df['Event'].iloc[n]
        start=pd.to_datetime(input_df['Start'].iloc[n])
        end=pd.to_datetime(input_df['End'].iloc[n])
        # Precipitation (3-month accumulated)
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(prec3, 'precipitation', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= '3-Month Precipitation'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        
        # Precipitation (6-month accumulated)
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(prec6, 'precipitation', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= '6-Month Precipitation'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # Minimum Daily Temperature
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(tmin, 'maximum daily air temperature', lat, lon, start, end,  option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'Min. Daily Temperature'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # Heat wave Index
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(hwi_edo, 'heatwave index', lat, lon, start, end,  option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'HWI-EDO'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # SPEI3
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(spei3, 'SPEI', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SPEI3'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        
        # SPEI6
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(spei6, 'SPEI', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SPEI6'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        
        # SPI12
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(spi12, 'SPI', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SPI12'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        
        # SPI6
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(spi6, 'SPI', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SPI6'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        
        # SPI3
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(spi3, 'SPI', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SPI3'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        
        # SSI3
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_data_pt(ssi3, 'SSI', 'time', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt 
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SSI3'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        
        # SSI6
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_data_pt(ssi6, 'SSI', 'time', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt 
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SSI6'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        # Precipitation (12-month accumulated)
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(prec12, 'precipitation', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= '12-Month Precipitation'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # Maximum Daily Temperature
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(tmax, 'maximum daily air temperature', lat, lon, start, end,  option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'Max. Daily Temperature'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # Heat wave Index
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(hwi, 'heatwave index', lat, lon, start, end,  option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'HWI'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # SPEI12
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(spei, 'SPEI', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SPEI12'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # Daily Flood Index
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_grid_pt(dfi, 'flood_index', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'DFI'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # SSI12
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_data_pt(ssi, 'SSI', 'time', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt 
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'SSI12'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # Discharge
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_data_pt(disc, 'value', 'date', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'Daily Discharge'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

        # Daily precipitation
        n_data_pt, data_pt, times_pt, lat_near, lon_near= extract_data_pt(dprec, 'value', 'ref', lat, lon, start, end, option)
        df_temp=pd.DataFrame(columns=['Indicator', 'Time', 'Value', 'Norm', 'Lat', 'Lon', 'Hazard', 'Event'])
        df_temp['Time']=times_pt
        df_temp['Value']=data_pt
        df_temp['Norm']=n_data_pt
        df_temp['Lat']=lat_near
        df_temp['Lon']=lon_near
        df_temp['Hazard']=haz
        df_temp['Indicator']= 'Daily Precipitation'
        df_temp['Event']=event
        df_results = pd.concat([df_results, df_temp], ignore_index=True)
        pbar.update()

  0%|          | 0/290 [00:00<?, ?it/s]

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

...

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt['norm'] = linear_min_max_normalization(df_pt[var].values)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_6036\933023062.py:89: SettingWithCopyWarning: 
A value is trying to be set on

In [59]:
# Functions to extract data under normal conditions
import random
from datetime import datetime, timedelta
# Function to generate random dates from January 1921 to December 2021
def generate_random_date():
    year = random.randint(1921, 2021)
    month = random.randint(1, 12)
    day = random.randint(1, 28)  # Assume the 28th day for simplicity
    #return f"{year:04d}-{month:02d}-{day:02d}"
    return datetime(year, month, day)

# Function to check if the conditions meet normal conditions
def check_normal_conditions(temp_max, spi12):
    return (np.nanmax(temp_max) < 25) and (-0.5 <= np.nanmean(spi12) <= 0.5)
def get_coordinates(municipality_name):
    geolocator = Nominatim(user_agent="municipality_geocoder")
    location = geolocator.geocode(municipality_name + ", Sweden")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None